# Arbeitspaket (AP) 2: Management & Nutzung Temporale Daten

### Persönliche Angaben (bitte ergänzen)

<table>
  <tr>
    <td>Vorname:</td>
    <td></td>
  </tr>
  <tr>
    <td>Nachname:</td>
    <td></td>
  </tr>
  <tr>
    <td>Immatrikulationsnummer:</td>
    <td></td>
  </tr>
  <tr>
    <td>Modul:</td>
    <td>Data Science</td>
  </tr>
  <tr>
    <td>Prüfungsdatum / Raum / Zeit:</td>
    <td>06.10.2025 / Raum: MU O2.001 / 8:00 – 11:45</td>
  </tr>
  <tr>
    <td>Erlaubte Hilfsmittel:</td>
    <td>w.MA.XX.DS.25HS (Data Science)<br>Open Book, Eigener Computer, Internet-Zugang</td>
  </tr>
  <tr>
  <td>Nicht erlaubt:</td>
  <td>Nicht erlaubt ist der Einsatz beliebiger Formen von generativer KI (z.B. Copilot, ChatGPT) <br> sowie beliebige Formen von Kommunikation oder Kollaboration mit anderen Menschen.</td>
</tr>
</table>

## Bewertungskriterien

### <b style="color: gray;">(max. erreichbare Punkte: 40)</b>

<table>
  <thead>
    <tr>
      <th>Kategorie</th>
      <th>Beschreibung</th>
      <th>Punkteverteilung</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Code nicht lauffähig oder Ergebnisse nicht sinnvoll</td>
      <td>Der Code enthält Fehler, die verhindern, dass er ausgeführt werden kann (z.B. Syntaxfehler) oder es werden Ergebnisse ausgegeben, welche nicht zur Fragestellung passen.</td>
      <td>0 Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit gravierenden Mängeln</td>
      <td>Der Code läuft, aber die Ergebnisse sind aufgrund wesentlicher Fehler unvollständig (z.B. fehlende Joins, gravierende Fehler in SQL-Abfragen). Nur geringer Fortschritt erkennbar.</td>
      <td>25% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit mittleren Mängeln</td>
      <td>Der Code läuft und liefert teilweise korrekte Ergebnisse, aber es gibt grössere Fehler (z.B. fehlende Spalten, unvollständige SQL-Abfragen). Die Ergebnisse sind nachvollziehbar, aber unvollständig oder ungenau.</td>
      <td>50% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit minimalen Mängeln</td>
      <td>Der Code läuft und liefert ein weitgehend korrektes Ergebnis, aber kleinere Fehler (z.B. falsche oder fehlende Sortierung, Rundung von Werten falsch) beeinträchtigen die Vollständigkeit des Ergebnisses.</td>
      <td>75% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig und korrekt</td>
      <td>Der Code läuft einwandfrei und liefert das korrekte Ergebnis ohne Mängel.</td>
      <td>100% der max. erreichbaren Punkte</td>
    </tr>
  </tbody>
</table>



## <b>Vorbereitung (Dieser Teil wird <u>nicht</u> bewertet!)</b>

#### <b>Beschreibung des Datensatzes `synthetic_ecommerce_orders.csv`</b>

- **Inhalt:** Synthetische E-Commerce-Bestellereignisse (April–Juni 2024)
- **Schema:**
  - `time` (lokale Zeit, Europe/Zurich, Format `YYYY-MM-DD HH:MM:SS`)
  - `customer_id` (int)
  - `product_id` (int)
  - `price` (float, EUR)
  - `discount` (float, EUR)
  - `status` (category: `placed`, `paid`, `cancelled`, `refunded`)
- **Hinweise zur Nutzung:**
  - Für **Abfragen** mit Tinyflux Zeitstempel nach **UTC** konvertieren.
  - `price - discount` kann als **Nettoerlös** interpretiert werden (v. a. für `paid`).
  - Leichte Tages-/Wochenmuster (mehr Aktivität 09:00–21:00, werktags).
- **Ziel:** Einlesen, in Tinyflux schreiben (Fields: `price`, `discount`; Tags: `customer_id`, `product_id`, `status`, optional `channel`), Zeitfenster-Abfragen und Aggregationen.


#### <b>1.) Python Libraries und Settings importieren:</b>

In [1]:
# Libraries
import os
import pandas as pd
import csv
from tinyflux import TinyFlux, Point, FieldQuery, TagQuery, TimeQuery
from datetime import datetime, timezone, timedelta

# Settings
import warnings
warnings.filterwarnings("ignore")

# Abkürzungen für Query-Typen
time = TimeQuery()
tags = TagQuery()
field = FieldQuery()

#### <b>2.) Funktion für die Erstellung eines Dataframes aus Query-Resultaten in den Arbeitsspeicher laden:</b>

In [2]:
# Define the function to transform the list of Point objects into a DataFrame
def points_to_dataframe(points_list):
    data = []
    
    # Extract the relevant information
    for point in points_list:
        data.append({
            "time": point.time,  # Access time attribute
            "customer_id": point.tags['customer_id'],  # Access customer_id from tags dictionary
            "product_id": point.tags['product_id'],  # Access product_id from tags dictionary
            "status": point.tags['status'],  # Access status from tags dictionary
            "price": point.fields['price'],  # Access price from fields dictionary
            "discount": point.fields['discount']  # Access discount from fields dictionary
        })

    # Create a DataFrame
    df = pd.DataFrame(data)

    # Convert time to datetime
    df['time'] = pd.to_datetime(df['time'])

    return df

#### <b>3.) Starten Sie eine GitHub Codespaces Instanz auf Basis Ihres Forks des folgenden GitHub Repositories:</b>

##### GitHub-Repository: https://github.com/MariaPelli/tinyflux


#### <b>4.) Erstellen Sie mit Tinyflux eine Datenbank mit dem Namen "ecommerce.db".</b>

In [3]:
# Erstellung der Tinyflux-Datenbank
db = TinyFlux("ecommerce.db")

## <b>Aufgaben: Einlesen der Daten in die Datenbank (Dieser Teil wird bewertet!)</b>

#### <b>Aufgabe (1): Lesen Sie die CSV-Datei 'synthetic_ecommerce_transactions.csv' in ein Dataframe mit dem Namen "df" ein, zeigen Sie die ersten 10 Records und geben deskriptive Statistiken des Dataframes aus.</b>

<b>Details zur Aufgabenstellung:</b>

- Das Laden der CSV-Datei kann mittels der Pandas-Funktion "read_csv" (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) erfolgen
- Achten Sie darauf, den Zeitstempel mittels der Pandas-Funktion __df['timestamp']= pd.to_datetime(df['timestamp'], format='mixed')__ auf das Format datetime zu setzen (https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)
- Die ersten 10 Records können mittels der Pandas-Funktion "df.head()" wiedergegeben werden (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html) 
- Deskriptive Statistiken können mittels Pandas-Funktion "df.describe()" (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) wiedergegeben werden. Geben Sie diese für quantitative und kategoriale Variablen aus, indem Sie den Parameter include auf 'all' setzen.

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [4]:
# CSV-Datei in Dataframe laden
df = pd.read_csv('synthetic_ecommerce_orders.csv')

# Das Format des Zeitstempels in Pandas setzen
df['time'] = pd.to_datetime(df['time'], format='mixed')

In [5]:
# Erste 10 Records des Dataframes anzeigen
df.head(10)

,time,customer_id,product_id,price,discount,status
0,2024-06-14 15:24:38,11192,686,29.07,5.52,paid
1,2024-05-07 22:45:57,10763,1246,39.82,0.00,cancelled
2,2024-06-04 19:16:06,10966,1332,15.97,1.28,paid
3,2024-04-15 22:58:24,12368,642,32.84,5.25,placed
4,2024-06-01 10:21:20,10847,1094,16.67,1.17,paid
5,2024-06-22 22:41:19,10360,1498,35.11,2.11,paid
6,2024-04-28 19:07:09,10130,674,23.90,2.87,cancelled
7,2024-06-07 08:35:33,12825,883,28.96,1.16,paid
8,2024-05-13 21:04:39,12776,960,18.69,3.18,paid
9,2024-06-10 12:55:15,10847,1230,14.73,2.80,paid


In [6]:
# Deskriptive Statistiken des Dataframes
df.describe(include='all')

,time,customer_id,product_id,price,discount,status
count,4000,4000.000000,4000.000000,4000.000000,4000.000000,4000
unique,NaN,NaN,NaN,NaN,NaN,4
top,NaN,NaN,NaN,NaN,NaN,paid
freq,NaN,NaN,NaN,NaN,NaN,2853
mean,2024-05-16 05:35:34.792000,11506.696750,998.228500,29.795737,3.025215,NaN
min,2024-04-01 00:17:20,10001.000000,500.000000,5.790000,0.000000,NaN
25%,2024-04-22 12:21:22.249999872,10750.000000,751.000000,20.047500,1.260000,NaN
50%,2024-05-15 23:07:57.500000,11519.000000,996.000000,26.925000,2.535000,NaN
75%,2024-06-08 11:09:16.249999872,12235.000000,1251.000000,36.532500,4.240000,NaN
max,2024-06-30 23:23:12,12999.000000,1499.000000,142.670000,18.360000,NaN


#### <b>Aufgabe (2): Schreiben Sie den Inhalt der CSV-Datei 'ecommerce.csv' in die erstellte Tinyflux-Datenbank "ecommerce.db" und weisen sie Time, Fields und Tags zu.</b>

<b style="color: gray;">(max. erreichbare Punkte: 8)</b>

In [10]:
# Einlesen der CSV-Datei in Tinyflux
with open('synthetic_ecommerce_orders.csv', mode="r") as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        point = Point(
            time=datetime.strptime(row['time'], '%Y-%m-%d %H:%M:%S'),
            measurement="ecommerce_orders",
            tags={
                "customer_id": row['customer_id'],
                "product_id": row['product_id'],
                "status": row['status']
            },
            fields={
                "price": float(row['price']),
                "discount": float(row['discount'])
            }
        )
    db.insert(point)

#### <b>Aufgabe (3): Die ersten drei Käufe erhalten am 7.7.2024 10% auf den am 1.4.2024 bezahlten Preis (ohne Berücksichtigung des Discounts) zurückerstattet. Fügen Sie die drei Transaktionen manuell in die Datenbank ein und weisen Sie Time, Fields und Tags gleichermassen zu, wie beim Inhalt der CSV-Datei.

</b>

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [ ]:
# Berechnung der Rückerstattung am 7.7.2024

v1 = 46.79 * 0.1
print('Rückerstattung 1. Kauf:', v1)
v2 = 22.37 * 0.1
print('Rückerstattung 2. Kauf:', v2)
v3 = 38.76 * 0.1
print('Rückerstattung 3. Kauf:', v3)

| time | customer_id | product_id | price | discount | status |
| --- | --- | --- | --- | --- | --- |
| 2024-07-07 00:00:00 | 12828 | 1114 | 4.679 | 0.00 | refunded |
| 2024-07-07 00:00:00 | 12313 | 1414 | 2.237 | 0.00 | refunded |
| 2024-07-07 00:00:00 | 12117 | 1242 | 3.876 | 0.00 | refunded |

In [11]:
# Einlesen von manuellen Records
p1 = Point(
    time=datetime(2024, 7, 7,0, 0, 0, tzinfo=timezone.utc),
    tags={"customer_id": "12828", "product_id": "1114", "status": "refunded"},
    fields={"price": 4.679, "discount": 0.4679 }
)
p2 = Point(
    time=datetime(2024, 7, 7,0, 0, 0, tzinfo=timezone.utc),
    tags={"customer_id": "12313", "product_id": "1414", "status": "refunded"},
    fields={"price": 2.237, "discount": 0.2237 }
)
p3 = Point(
    time=datetime(2024, 7, 7,0, 0, 0, tzinfo=timezone.utc),
    tags={"customer_id": "12117", "product_id": "1242", "status": "refunded"},
    fields={"price": 3.876, "discount": 0.3876 }
)
db.insert_multiple([p1, p2, p3])  

3

<div>

#### <b> Hinweis! 

#### Falls etwas mit der Erstellung der Datenbank nicht funktioniert, verwenden Sie für die folgenden Aufgaben die Tinyflux Datenbank 'ecommerce_fallback.db'. Gehen Sie davon aus, dass diese unter denselben, obigen Bedingungen erstellt wurde.</b>
    
#### <b>Erstellen der Verbindung mit der Tinyflux-Datenbank mit dem Namen "ecommerce_fallback.db":</b>
</div>

In [ ]:
# Erstellung der Tinyflux-Datenbank (wird bereitgestellt)
db = TinyFlux("ecommerce_fallback.db")

In [12]:
# Erstellung der Tinyflux-Datenbank (wird bereitgestellt)
db = TinyFlux("ecommerce.db")

## <b>Aufgaben: Abfragen (Queries) (Dieser Teil wird bewertet!)</b>

<b >Hinweise zu den folgenden Aufgabenstellungen:</b>
<ul>
  <li>Erstellen Sie je Aufgabe eine Time-, Field- oder Tag Abfrage bzw. eine Kombination von diesen und integrieren Sie diese in Ihren Python Code.</li>
  <li>Speichern Sie je Aufgabe die Ergebnistabelle mittels der zu Beginn dieses Notebooks bereitgestellten Funktion <b>points_to_dataframe</b> in einem DataFrame und stellen Sie diesen bzw. dessen Eigenschaften ganz oder in Teilen (z.B. erste 5 Zeilen) dar.</li>
</ul>

#### <b>Aufgabe (4): Schreiben Sie alle Bestellungen in ein Dataframe, die bis und mit dem 4.4.2024 getätigt wurden und geben Sie die ersten 10 Records dieses Dataframes aus</b>
<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [13]:
from tinyflux import TimeQuery
from datetime import datetime, timezone
time = TimeQuery()

In [18]:

# Erstellung eines Dataframes aus den Query-Resultaten und Ausgabe der ersten 10 Records

# Definieren der Query
time_query_conditions = (time <= datetime(2024, 4, 4, tzinfo=timezone.utc))

# Datenabfrage mittels der definierten Query
my_results = db.search(time_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
df_result = points_to_dataframe(my_results)

#Schritt 1: Entfernen von Duplikaten (z.B. nach Zeitstempel)
df_unique = df_result.drop_duplicates(subset=["time"])

#Schritt 2: Sortieren nach Zeitstempel
df_last10 = df_unique.sort_values("time", ascending=False).head(10)
df_last10

,time,customer_id,product_id,status,price,discount
158,2024-04-03 23:10:17+00:00,12488,1244,paid,40.40,0.00
157,2024-04-03 22:03:30+00:00,11216,1069,paid,15.26,2.75
156,2024-04-03 20:59:44+00:00,12106,1129,paid,47.73,1.91
155,2024-04-03 20:56:43+00:00,12171,1079,paid,70.07,7.71
154,2024-04-03 20:27:17+00:00,12021,755,paid,25.13,1.26
153,2024-04-03 20:15:42+00:00,11731,519,paid,22.37,4.47
152,2024-04-03 19:45:33+00:00,10735,898,paid,19.03,1.90
151,2024-04-03 18:58:28+00:00,12930,513,paid,48.86,3.91
150,2024-04-03 18:53:39+00:00,11557,1045,cancelled,16.29,0.98
149,2024-04-03 18:50:18+00:00,12076,1041,paid,17.57,1.23


#### <b>Aufgabe (5): Ermitteln Sie über den gesamten Datensatz die Anzahl Transaktionen pro Monat</b>

<b>Details zur Aufgabenstellung:</b>
- Die Anzahl Transaktionen kann mittels len(df_result), angewendet auf das Dataframe oder die Rohausgabe des Queries, herausgegeben werden

<b style="color: gray;">(max. erreichbare Punkte: 8)</b>

In [20]:
# Zeige alle Zeitstempel aus dem DataFrame
print(df['time'].min(), df['time'].max())
print(df['time'].dt.year.value_counts())
print(df[df['time'].dt.year == 2024]['time'].dt.month.value_counts())

2024-04-01 00:17:20 2024-06-30 23:23:12
time
2024    4000
Name: count, dtype: int64
time
5    1339
4    1338
6    1323
Name: count, dtype: int64


In [26]:
# Definieren der Queries
time_query_conditions_Jan = (time >= datetime(2024, 1, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 1, 31, tzinfo=timezone.utc))
time_query_conditions_Feb = (time >= datetime(2024, 2, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 2, 28, tzinfo=timezone.utc))
time_query_conditions_Mar = (time >= datetime(2024, 3, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 3, 31, tzinfo=timezone.utc))
time_query_conditions_Apr = (time >= datetime(2024, 4, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 4, 30, tzinfo=timezone.utc))
time_query_conditions_May = (time >= datetime(2024, 5, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 5, 31, tzinfo=timezone.utc))
time_query_conditions_Jun = (time >= datetime(2024, 6, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 6, 30, tzinfo=timezone.utc))
time_query_conditions_Jul = (time >= datetime(2024, 7, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 7, 31, tzinfo=timezone.utc))
time_query_conditions_Aug = (time >= datetime(2024, 8, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 8, 31, tzinfo=timezone.utc))
time_query_conditions_Sep = (time >= datetime(2024, 9, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 9, 30, tzinfo=timezone.utc))
time_query_conditions_Oct = (time >= datetime(2024, 10, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 10, 31, tzinfo=timezone.utc))
time_query_conditions_Nov = (time >= datetime(2024, 11, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 11, 30, tzinfo=timezone.utc))
time_query_conditions_Dec = (time >= datetime(2024, 12, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 12, 31, tzinfo=timezone.utc))

# Datenabfrage mittels der definierten Queries
my_results_Jan = db.search(time_query_conditions_Jan)
my_results_Feb = db.search(time_query_conditions_Feb)
my_results_Mar = db.search(time_query_conditions_Mar)
my_results_Apr = db.search(time_query_conditions_Apr)
my_results_May = db.search(time_query_conditions_May)
my_results_Jun = db.search(time_query_conditions_Jun)
my_results_Jul = db.search(time_query_conditions_Jul)
my_results_Aug = db.search(time_query_conditions_Aug)
my_results_Sep = db.search(time_query_conditions_Sep)
my_results_Oct = db.search(time_query_conditions_Oct)
my_results_Nov = db.search(time_query_conditions_Nov)
my_results_Dec = db.search(time_query_conditions_Dec)

# Erstellung eines Dataframes aus den Query-Resultaten
df_result_Jan = points_to_dataframe(my_results_Jan)
df_result_Feb = points_to_dataframe(my_results_Feb)
df_result_Mar = points_to_dataframe(my_results_Mar)
df_result_Apr = points_to_dataframe(my_results_Apr)
df_result_May = points_to_dataframe(my_results_May)
df_result_Jun = points_to_dataframe(my_results_Jun)
df_result_Jul = points_to_dataframe(my_results_Jul)
df_result_Aug = points_to_dataframe(my_results_Aug)
df_result_Sep = points_to_dataframe(my_results_Sep)
df_result_Oct = points_to_dataframe(my_results_Oct)
df_result_Nov = points_to_dataframe(my_results_Nov)
df_result_Dec = points_to_dataframe(my_results_Dec)

# Ausgabe der aggregierten Resultate
print("\nTime Query Count January:")
print(len(df_result_Jan))
print("\nTime Query Count February:")
print(len(df_result_Feb))
print("\nTime Query Count March:")
print(len(df_result_Mar))
print("\nTime Query Count April:")
print(len(df_result_Apr))
print("\nTime Query Count May:")
print(len(df_result_May))
print("\nTime Query Count June:")
print(len(df_result_Jun))
print("\nTime Query Count July:")
print(len(df_result_Jul))
print("\nTime Query Count August:")
print(len(df_result_Aug))
print("\nTime Query Count September:")
print(len(df_result_Sep))
print("\nTime Query Count October:")
print(len(df_result_Oct))
print("\nTime Query Count November:")
print(len(df_result_Nov))
print("\nTime Query Count December:")
print(len(df_result_Dec))

KeyError: 'time'

#### <b>Aufgabe (6): Ermitteln Sie den Median aller Discount-Beträge pro Status und Monat</b>

<b>Details zur Aufgabenstellung:</b>
- Alle im Datensatz vorhandenen Status können Sie mittels der Pandas-Funktion "df.attributname.unique()" (https://pandas.pydata.org/docs/reference/api/pandas.unique.html) ermitteln
- Den Durchschnitt über eine Spalte können Sie mittels der Pandas-Funktion df[['spaltenname']].mean() https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mean.html ermitteln

<b style="color: gray;">(max. erreichbare Punkte: 12)</b>

In [24]:
df.status.unique()

array(['paid', 'cancelled', 'placed', 'refunded'], dtype=object)

In [40]:
# Definieren der Time-Queries
time_query_conditions_m4 = (time >= datetime(2024, 4, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 4, 30, tzinfo=timezone.utc))
time_query_conditions_m5 = (time >= datetime(2024, 5, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 5, 31, tzinfo=timezone.utc))
time_query_conditions_m6 = (time >= datetime(2024, 6, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 6, 30, tzinfo=timezone.utc))
time_query_conditions_m7 = (time >= datetime(2024, 7, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 7, 31, tzinfo=timezone.utc))

field_query_conditions = (field.fee > 0.050444)

# Definieren der Tag-Queries
tag_query_conditions_paid = (tags.status == "paid")
tag_query_conditions_cancelled = (tags.status == "cancelled")
tag_query_conditions_placed = (tags.status == "placed")
tag_query_conditions_refunded = (tags.status == "refunded")

# Datenabfrage mittels der definierten Queries
my_resulats_paid = db.search(tag_query_conditions_paid & time_query_conditions_m4)
my_resulats_cancelled = db.search(tag_query_conditions_cancelled & time_query_conditions_m4)
my_resulats_placed = db.search(tag_query_conditions_placed & time_query_conditions_m4)
my_resulats_refunded = db.search(tag_query_conditions_refunded & time_query_conditions_m4)

# Erstellung eines Dataframes aus den Query-Resultaten
df_result_paid = points_to_dataframe(my_resulats_paid)
df_result_cancelled = points_to_dataframe(my_resulats_cancelled)
df_result_placed = points_to_dataframe(my_resulats_placed)
df_result_refunded = points_to_dataframe(my_resulats_refunded)

# Ausgabe der aggregierten Resultate

# - paid, April
print ("\nMedian für paid:")
print(df_result_paid[['paid']].mean())
# - placed, April
print ("\nMedian für placed:")
print(df_result_placed[['placed']].mean())
# - refunded, April
print ("\nMedian für refuned:")
print(df_result_cancelled[['refunded']].mean())
# - cancelled, April
print ("\nMedian für cancelled:")
print(df_result_cancelled[['cancelled']].mean())


# - paid, Mai

# - placed, Mai

# - refunded, Mai

# - cancelled, Mai



# - paid, Juni

# - placed, Juni

# - refunded, Juni

# - cancelled, Juni




Median für paid:


KeyError: "None of [Index(['paid'], dtype='object')] are in the [columns]"

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('IP Address:', socket.gethostbyname(socket.gethostname()))
print('-----------------------------------')